In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics

from sklearn.linear_model import ElasticNet,ElasticNetCV


from warnings import simplefilter

import pickle
from sklearn.externals import joblib

rand_st=4
#simplefilter("ignore") 

In [2]:
#output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx')
output=pd.read_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results_robust.xlsx')
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",342.986713,431.864722,0.458002,0.175,Обучение на сумме рейтингов
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.724529,192.311526,0.574081,0.194,Обучение на разнице в рейтинге


In [29]:
#elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos=pd.read_csv(r'C:\Users\vznam\Downloads\PredictingRatings-master\data\clean_elos.csv')
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,2354,2411,2382,-57,4765
1,1915,1999,1957,-84,3914
2,2446,2191,2318,255,4637
3,2168,2075,2121,93,4243
4,2437,2254,2345,183,4691


In [20]:
from sklearn.preprocessing import RobustScaler

robust = RobustScaler(quantile_range = (0.1,0.9))
frobust=robust.fit_transform(elos)
elos = pd.DataFrame(frobust,columns=elos.columns)
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,0.383033,0.666020,0.647295,-0.309513,0.651336
1,-1.618772,-1.200649,-1.664473,-0.436132,-1.667857
2,0.802546,-0.330745,0.299170,1.153641,0.302503
3,-0.465112,-0.856312,-0.772402,0.393926,-0.771247
4,0.761507,-0.045308,0.446035,0.815990,0.449667


In [30]:
#features=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv')
features=pd.read_csv(r'C:\Users\vznam\Downloads\PredictingRatings-master\data\features_robust.csv')

print(features.info())
print(features.shape)
features.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24255 entries, 0 to 24254
Columns: 224 entries, 0 to 223
dtypes: float64(224)
memory usage: 41.5 MB
None
(24255, 224)


,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
count,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,...,24255.000000,24255.000000,24255.000000,24255.00000,24255.000000,24255.000000,24255.00000,24255.000000,24255.000000,24255.000000
mean,0.044877,0.750897,0.009597,0.002444,0.086345,0.827871,-0.834333,46.920092,3.124728,40.735301,...,0.479365,0.465512,0.467697,0.07384,0.472727,-0.368707,0.03892,1.015584,1.457308,0.910740
std,0.417736,5.411864,0.355022,0.091364,1.783087,5.414979,2.465112,128.245270,10.069143,114.569437,...,0.619074,0.502279,0.611065,1.83264,0.502640,0.676858,0.74847,2.874868,4.179615,3.663225
min,-0.500000,-10.000000,-3.321955,-2.349066,-9.460765,-10.000000,-14.229961,-11.664600,-7.688687,-33.209975,...,0.000000,0.000000,0.000000,-4.00000,0.000000,-1.000000,-1.00000,-1.000000,0.000000,0.000000
25%,-0.500000,-3.000000,-0.129611,0.000000,-0.279324,-3.000000,-0.471897,-5.015215,-2.981327,-20.309038,...,0.000000,0.000000,0.000000,-1.00000,0.000000,-1.000000,-1.00000,-1.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,0.500000,3.666667,0.151033,0.005336,0.421687,3.833333,0.066089,30.654795,5.491919,35.106446,...,1.000000,1.000000,1.000000,1.00000,1.000000,0.000000,1.00000,2.000000,0.000000,0.000000
max,0.500000,41.666667,3.112170,2.310486,9.519408,41.666667,0.117105,687.253466,115.330300,919.718217,...,2.000000,2.000000,2.000000,4.00000,2.000000,1.000000,1.00000,70.000000,35.000000,36.000000


In [31]:
target_white=elos['WhiteElo']
target_black=elos['BlackElo']
target_mean=elos['MeanElos']
target_diff=elos['DiffElos']
target_sum=elos['SumElos']

In [32]:
X_train_sum, X_test_sum, Y_train_sum, Y_test_sum =ms.train_test_split(features, target_sum, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_sum), len(Y_train_sum)))
print(X_train_sum.shape)
print(Y_train_sum.shape)
print(X_test_sum.shape)
print(Y_test_sum.shape)

Train data size: 16978 instances 
Test data size: 16978 instances
(16978, 224)
(16978,)
(7277, 224)
(7277,)


In [33]:
X_train_diff, X_test_diff, Y_train_diff, Y_test_diff =ms.train_test_split(features, target_diff, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_diff), len(Y_train_diff)))
print(X_train_diff.shape)
print(Y_train_diff.shape)
print(X_test_diff.shape)
print(Y_test_diff.shape)

Train data size: 16978 instances 
Test data size: 16978 instances
(16978, 224)
(16978,)
(7277, 224)
(7277,)


#### Для теста

In [34]:
target_sum1=target_sum[:5000]
target_diff1=target_diff[:5000]
features1=features[:5000]

X_train1, X_test1, Y_train1, Y_test1 =ms.train_test_split(features1, target_sum1, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train1), len(Y_train1)))
print(X_train1.shape)
print(Y_train1.shape)
print(X_test1.shape)
print(Y_test1.shape)

Train data size: 3500 instances 
Test data size: 3500 instances
(3500, 224)
(3500,)
(1500, 224)
(1500,)


In [35]:
X_train2, X_test2, Y_train2, Y_test2 =ms.train_test_split(features1, target_diff1, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train2), len(Y_train2)))
print(X_train2.shape)
print(Y_train2.shape)
print(X_test2.shape)
print(Y_test2.shape)

Train data size: 3500 instances 
Test data size: 3500 instances
(3500, 224)
(3500,)
(1500, 224)
(1500,)


In [36]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

In [37]:
def GridTune_sum(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
   
    grid = ms.GridSearchCV(classifiers,
                           params,
                           refit=True,  
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))

    
    grid_best_model = grid.fit(X_train1,Y_train1)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

#### ElasticNet

In [38]:
elastic_sum_params =[{'l1_ratio':[0,0.25,0.5,0.75,1],'alpha':[1,0.5],'fit_intercept':[False]},
                     {'l1_ratio':[0,0.25,0.5,0.75,1],'alpha':[1,0.5],'fit_intercept':[True],'normalize':[True,False]}]

start_elastic_sum = time.time()

elastic_sum=ElasticNet(random_state=rand_st)
elastic_sum_model=GridTune_sum(elastic_sum_params,elastic_sum,5)
elastic_sum_predictions=elastic_sum_model.predict(X_test1)

scores_elastic_sum=predictions_report(Y_test1,elastic_sum_predictions)
full_time_elastic_sum=round(time.time() - start_elastic_sum,3)
print(full_time_elastic_sum, 'c')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Лучшие параметры для MAE:
{'alpha': 1, 'fit_intercept': False, 'l1_ratio': 0}

Лучший алгоритм:
ElasticNet(alpha=1, copy_X=True, fit_intercept=False, l1_ratio=0,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=4, selection='cyclic', tol=0.0001, warm_start=False)
MAE = 1031.830
RMSE = 1325.733
Correlation coefficient = 0.052
27.658 c


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [ ]:
start_elastic_sum = time.time()

elastic_sum=ElasticNet(alpha=1, copy_X=True, fit_intercept=False, l1_ratio=0,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=4, selection='cyclic', tol=0.0001, warm_start=False).fit(X_train_sum,Y_train_sum)
elastic_sum_predictions=elastic_sum.predict(X_test_sum)

scores_elastic_sum=predictions_report(Y_test_sum,elastic_sum_predictions)
full_time_elastic_sum=round(time.time() - start_elastic_sum,3)
print(full_time_elastic_sum, 'c')

In [ ]:
output = output.append(pd.Series([str(elastic_sum), scores_elastic_sum[0],scores_elastic_sum[1], 
                                  scores_elastic_sum[2][0],full_time_elastic_sum,'ElasticNet по сумме рейтингов'],
                                 index=output.columns ),
                      ignore_index=True)

output

In [ ]:
#output.to_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx',index=False)
output.to_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results.xlsx',index=False)

In [ ]:
with open('elastic_sum.pkl', 'wb') as f:
    pickle.dump(elastic_sum,f) 
                         
'''rf_sum = pickle.loads(model_rf_sum)
rf_sum.predict(X[0])'''

joblib.dump(elastic_sum, 'elastic_sum1.pkl')
#model_sum= joblib.load('filename.pkl') 

In [ ]:
def GridTune_diff(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
   
    grid = ms.GridSearchCV(classifiers,
                           params,
                           refit=True,  
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))
    
    grid_best_model = grid.fit(X_train2,Y_train2)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

In [ ]:
elastic_diff_params =[{'l1_ratio':[0,0.25,0.5,0.75,1],'alpha':[1,0.5],'fit_intercept':[False]},
                     {'l1_ratio':[0,0.25,0.5,0.75,1],'alpha':[1,0.5],'fit_intercept':[True],'normalize':[True,False]}]

start_elastic_diff = time.time()

elastic_diff=ElasticNet(random_state=rand_st)
elastic_diff_model=GridTune_diff(elastic_diff_params,elastic_diff,5)
elastic_diff_predictions=elastic_diff_model.predict(X_test2)

scores_elastic_diff=predictions_report(Y_test2,elastic_diff_predictions)
full_time_elastic_diff=round(time.time() - start_elastic_diff,3)
print(full_time_elastic_diff, 'c')

In [ ]:
start_elastic_diff = time.time()

elastic_diff=ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=1,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=4, selection='cyclic', tol=0.0001, warm_start=False).fit(X_train_diff,Y_train_diff)
elastic_diff_predictions=elastic_diff.predict(X_test_diff)

scores_elastic_diff=predictions_report(Y_test_diff,elastic_diff_predictions)
full_time_elastic_diff=round(time.time() - start_elastic_diff,3)
print(full_time_elastic_diff, 'c')

In [ ]:
output = output.append(pd.Series([str(elastic_diff), scores_elastic_diff[0],scores_elastic_diff[1], 
                                  scores_elastic_diff[2][0],full_time_elastic_diff,'ElasticNet на разнице в рейтинге'],
                                 index=output.columns ),
                      ignore_index=True)

output

In [ ]:
#output.to_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx',index=False)
output.to_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results.xlsx',index=False)

In [ ]:
with open('elastic_diff.pkl', 'wb') as f:
    pickle.dump(elastic_diff,f) 
                         
'''rf_sum = pickle.loads(model_rf_sum)
rf_sum.predict(X[0])'''

joblib.dump(elastic_diff, 'elastic_diff1.pkl')
#model_sum= joblib.load('filename.pkl') 

#### ElasticNetCV

In [ ]:
cvelastic_sum_params =[{'l1_ratio':[0.25,0.5,0.75,1],'n_alphas':[1,2,3],'fit_intercept':[False]},
                     {'l1_ratio':[0.25,0.5,0.75,1],'n_alphas':[1,2,3],'fit_intercept':[True],'normalize':[True,False]}]

start_cvelastic_sum = time.time()

cvelastic_sum=ElasticNetCV(random_state=rand_st,n_jobs=-1,cv=10)
cvelastic_sum_model=GridTune_sum(cvelastic_sum_params,cvelastic_sum,5)
cvelastic_sum_predictions=cvelastic_sum_model.predict(X_test1)

scores_cvelastic_sum=predictions_report(Y_test1,cvelastic_sum_predictions)
full_time_cvelastic_sum=round(time.time() - start_cvelastic_sum,3)
print(full_time_cvelastic_sum, 'c')

In [ ]:
start_cvelastic_sum = time.time()

cvelastic_sum=ElasticNetCV(copy_X=True, fit_intercept=False,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=4, selection='cyclic', tol=0.0001, n_jobs=-1,cv=10).fit(X_train_sum,Y_train_sum)
cvelastic_sum_predictions=cvelastic_sum.predict(X_test_sum)

scores_cvelastic_sum=predictions_report(Y_test_sum,cvelastic_sum_predictions)
full_time_cvelastic_sum=round(time.time() - start_cvelastic_sum,3)
print(full_time_cvelastic_sum, 'c')

ElasticNetCv похоже вообще не сходится

#### Проверки

In [ ]:
white_elos = (elastic_sum_predictions + elastic_diff_predictions)/2
black_elos = (elastic_sum_predictions-elastic_diff_predictions)/2
pred_elos=list(zip(white_elos,black_elos))
pred_elos[:5],elos[:5]

In [ ]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=elos.iloc[ind,0]-val[0]
        er+=elos.iloc[ind,1]-val[1]
        count+=2
    return round(er/count,5)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=elos.iloc[ind,0]-val[0]
        er+=elos.iloc[ind,1]-val[1]
    return round(er,5)

In [ ]:
find_abs_error(pred_elos,elos),find_mean_error(pred_elos,elos)